In [2]:
import requests
import time
import csv

# Define constants
API_KEY = 'YOUR_ETHERSCAN_API_KEY'  # Replace with your Etherscan API key
WALLET_ADDRESS = '0xd4bC53434C5e12cb41381A556c3c47e1a86e80E3'
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
START_BLOCK = 0  # Starting block (could be set to a higher value to narrow down the search)
END_BLOCK = 99999999  # Use a high block number for the latest block

OFFSET_LENGTH = 2500

def get_transactions(wallet_address, start_block=START_BLOCK, end_block=END_BLOCK):
    transactions = []
    page = 1  # Start with page 1
    while True:
        # Call Etherscan API to get transactions for the wallet
        params = {
            'module': 'account',
            'action': 'txlist',
            'address': wallet_address,
            'startblock': start_block,
            'endblock': end_block,
            'page': page,
            'offset': OFFSET_LENGTH,  # Number of transactions per page
            'sort': 'desc',  # Sort by descending block number
            'apikey': API_KEY
        }

        response = requests.get(ETHERSCAN_API_URL, params=params)
        data = response.json()

        if data['status'] == '1' and 'result' in data:
            transactions.extend(data['result'])
            print(len(transactions))

            # If there are more results, move to the next page
            if len(data['result']) == OFFSET_LENGTH:
                page += 1
            else:
                break  # All results retrieved
        else:
            print(f"Error: {data.get('message', 'Unknown error')}")
            break

        # Respect rate limits (about 5 requests per second)
        time.sleep(1)

    return transactions

def save_transactions_to_csv(transactions, filename='transactions.csv'):
    # Define the CSV header
    header = ['BlockNumber', 'TimeStamp', 'Hash', 'From', 'To', 'Value', 'Gas', 'GasPrice', 'IsError', 'TxReceiptStatus']
    
    # Open CSV file for writing
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()  # Write the header row
        for transaction in transactions:
            writer.writerow({
                'BlockNumber': transaction.get('blockNumber'),
                'TimeStamp': transaction.get('timeStamp'),
                'Hash': transaction.get('hash'),
                'From': transaction.get('from'),
                'To': transaction.get('to'),
                'Value': transaction.get('value'),
                'Gas': transaction.get('gas'),
                'GasPrice': transaction.get('gasPrice'),
                'IsError': transaction.get('isError'),
                'TxReceiptStatus': transaction.get('txReceiptStatus')
            })

    print(f"Data saved to {filename}")

# Fetch transactions
transactions = get_transactions(WALLET_ADDRESS)

# Print the results and save them to a CSV file
if transactions:
    print(f"Found {len(transactions)} transactions for wallet {WALLET_ADDRESS}")
    save_transactions_to_csv(transactions)
else:
    print("No transactions found.")

2500
5000
7500
10000
Error: Result window is too large, PageNo x Offset size must be less than or equal to 10000
Found 10000 transactions for wallet 0xd4bC53434C5e12cb41381A556c3c47e1a86e80E3
Data saved to transactions.csv


In [8]:
import pandas as pd

# Load the CSV file containing the ERC-20 token transfers
file_path = 'transactions.csv'  # Make sure the path is correct if not in the same directory
df = pd.read_csv(file_path, index_col='Hash')


# Convert 'TimeStamp' from Unix timestamp to yyyy-mm-dd hh:mm:ss format
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M')

# Rename 'Value' to 'Value_ETH' to represent Ether values
df.rename(columns={'Value': 'Value_Eth'}, inplace=True)
df.rename(columns={'GasPrice': 'GasPrice_Gwei'}, inplace=True)

# Convert 'Value_ETH' from Wei to Ether 
df['Value_Eth'] = pd.to_numeric(df['Value_Eth'], errors='coerce') / 10**18  # Convert Wei to Ether

# Convert GasPrice from Wei to Gwei and Ether
df['GasPrice_Gwei'] = pd.to_numeric(df['GasPrice_Gwei'], errors='coerce') / 10**9

# Calculate Transaction Cost in Ether
df['TransactionCost_Eth'] = df['Gas'] * df['GasPrice_Gwei'] * 10**-9 * 0.7

# Save the modified DataFrame to the same CSV file
df.to_csv('./csv/transactions.csv')

In [9]:
df

,BlockNumber,TimeStamp,From,To,Value_Eth,Gas,GasPrice_Gwei,IsError,TxReceiptStatus,TransactionCost_Eth
Hash,,,,,,,,,,
0x54b7b4b8eba70b292b2f2580cbcc0b3f55d58beb31ec938606b5afb97fc94dac,21457908,2024-12-22 12:02,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,2.070000e-16,773040,38.326654,0,NaN,0.020740
0x960355990a7e3926e7c119c0404308da157d4d6f1b2edc18a5ab630cfa2e681f,21457905,2024-12-22 12:01,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,1.710000e-16,632952,63.906316,0,NaN,0.028315
0xa78eb4463b0b19fa2b4b6a5ee94de7c3acc7639f2d257838e219364712a8edf4,21457896,2024-12-22 11:59,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,6.300000e-17,149784,7.301182,0,NaN,0.000766
0x9a5a4e7b1c0598728d4fcb4b0616cc39052f23a5361c2781f8b7f4f7cc520c00,21457893,2024-12-22 11:59,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,2.700000e-17,360028,7.971582,0,NaN,0.002009
0xceb7b5b7d5750356e2a278437d40c34b449f367178b212560eb462b40dcfbbdb,21457886,2024-12-22 11:57,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,1.990000e-16,274717,109.359771,0,NaN,0.021030
...,...,...,...,...,...,...,...,...,...,...
0x6157374d79e2d39d6cec1b0168ba22547b6a61de0a0a4450baafef2d1b5a7c72,21394062,2024-12-13 14:02,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,1.470000e-16,412365,17.846769,0,NaN,0.005152
0x9514a2c7b723cb5f832cdfa62a6987adcde0c3f3c72d0ab4b53f2cba7a956eaf,21394055,2024-12-13 14:01,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,6.300000e-17,204324,19.555346,0,NaN,0.002797
0x9bfbba1aecf0e4d25b1ec4b21c1153b80fcc72ea68cd76843890ffcf27c51a13,21394053,2024-12-13 14:01,0xd1fa51f2db23a9fa9d7bb8437b89fb2e70c60cb7,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,3.900000e-17,271027,31.992485,0,NaN,0.006070
